<a href="https://colab.research.google.com/github/emmatliu/spuco-mnist/blob/main/spuco_mnist_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install spuco

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.0/101.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.2/126.2 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 101.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 103.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 19.7 MB/s eta 0:00:00
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7028 sha256=2cb392bb3c3eb8fe8655eedce08e58038bc6376f08426bcf611c15e3fcd9a940
  Stored in directory: /root/.cache/pip/wheels/3d/fe/b0/27a9892da57472e538c7452a721a9cf463cc03cf7379889266
Successfully built littleutils


In [ ]:
import spuco
import torch

from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
from spuco.robust_train import ERM, GroupBalanceBatchERM
from spuco.evaluate import Evaluator
from spuco.models import model_factory
from spuco.group_inference import Cluster, ClusterAlg
from spuco.utils import Trainer

In [ ]:
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_LARGE
classes = [[1,7],[2,5],[3,8],[4,9],[0,6]] # Visual similarity?

train = SpuCoMNIST(
    "/mnist/",
    difficulty,
    classes,
    0.99,
    split="train"
)

test = SpuCoMNIST(
    "/mnist/",
    difficulty,
    classes,
    split="test"
)

train.initialize()
test.initialize()

100%|██████████| 10000/10000 [00:02<00:00, 4444.95it/s]


In [ ]:
train_shape = train[0][0].shape

# Step 1: Train a model using ERM

In [ ]:
model = model_factory('lenet',train_shape,len(classes))
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
erm = ERM(
    model=model,
    trainset=train,
    batch_size=64,
    optimizer=optimizer,
    num_epochs=5,
    verbose=True
)
erm.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Epoch 4: 100%|██████████| 751/751 [00:26<00:00, 28.09batch/s, accuracy=100.0%, loss=0.0162]


# Step 2: Cluster inputs using ERM output

In [ ]:
trainer = Trainer(
    trainset=train,
    model=model,
    batch_size=64,
    optimizer=optimizer,
    verbose=True
)
trainer.train(0)

outs = trainer.get_trainset_outputs()

cluster = Cluster(
    Z=outs,
    class_labels=train.labels,
    num_clusters=3,
    verbose=True
)
groups = cluster.infer_groups()

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Clustering class-wise: 100%|██████████| 5/5 [00:00<00:00,  9.31it/s]


# Step 3: Retrain using group-balancing

In [ ]:
gb_erm = GroupBalanceBatchERM(
    model=model,
    num_epochs=5,
    trainset=train,
    group_partition=groups,
    batch_size=64,
    optimizer=optimizer,
    verbose=True
)
gb_erm.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Epoch 4: 100%|██████████| 751/751 [00:16<00:00, 46.43batch/s, accuracy=100.0%, loss=0.000121]


# Some evaluation

In [ ]:
evaluator = Evaluator(
    testset=test,
    group_partition=test.group_partition,
    group_weights=test.group_weights,
    batch_size=64,
    model=model,
    verbose=True
)

evaluator.evaluate()

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Evaluating group-wise accuracy:   4%|▍         | 1/25 [00:00<00:13,  1.79it/s]

Group (0, 0) Accuracy: 99.7690531177829


Evaluating group-wise accuracy:   8%|▊         | 2/25 [00:01<00:11,  1.93it/s]

Group (0, 1) Accuracy: 52.19399538106236


Evaluating group-wise accuracy:  12%|█▏        | 3/25 [00:01<00:09,  2.31it/s]

...

Evaluating group-wise accuracy:  96%|█████████▌| 24/25 [00:09<00:00,  2.15it/s]

Group (4, 3) Accuracy: 81.65374677002583


Evaluating group-wise accuracy: 100%|██████████| 25/25 [00:10<00:00,  2.49it/s]

Group (4, 4) Accuracy: 100.0


{(0, 0): 99.7690531177829,
 (0, 1): 52.19399538106236,
 (0, 2): 42.263279445727484,
 (0, 3): 75.23148148148148,
 (0, 4): 93.98148148148148,
 (1, 0): 32.72727272727273,
 (1, 1): 99.48051948051948,
 (1, 2): 14.025974025974026,
 (1, 3): 36.62337662337662,
 (1, 4): 45.052083333333336,
 (2, 0): 71.53652392947103,
 (2, 1): 75.56675062972292,
 (2, 2): 100.0,
 (2, 3): 66.49874055415617,
 (2, 4): 62.121212121212125,
 (3, 0): 74.93734335839599,
 (3, 1): 85.92964824120602,
 (3, 2): 74.37185929648241,
 (3, 3): 100.0,
 (3, 4): 85.17587939698493,
 (4, 0): 96.64948453608247,
 (4, 1): 85.82474226804123,
 (4, 2): 87.37113402061856,
 (4, 3): 81.65374677002583,
 (4, 4): 100.0}

In [ ]:
evaluator.evaluate_spurious_attribute_prediction()

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


41.22